# 9 WorkFlow Analista Jr

### 9.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán
<br>El Analista Jr corre sus scripts en la virtual manchine **desktop-jr** que tiene estas características


*   Normal, paga tarifa completa, nunca es apagada por Google
*   reside en el datacenter de Sao Paulo, Brasil
*   64 GB de memoria RAM
*   8 vCPU



## 9.3  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [10]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sat Nov 29 20:21:45 2025"

In [11]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2084217,111.4,4088337,218.4,2828802,151.1
Vcells,3603551,27.5,78009248,595.2,60730998,463.4


In [12]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")


#### Parametros

In [13]:
PARAM <- list()
PARAM$semilla_primigenia <- 943153

PARAM$experimento <- "WF_semillero_50bayes_FE_kaggle_final"
PARAM$dataset <- "analistajr_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [14]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 9.3.1   Preprocesamiento del dataset

#### 9.3.1.1  DT incorporar dataset

In [15]:
# lectura del dataset
dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

#### 9.3.1.2  CA  Catastrophe Analysis
Se intentan reparar las variables que para un mes están con todos los valores en cero.

El método que se utiliza es **Machine Learning** se asigna NA also valores, si ha leido bien, es la "anti imputación de valores faltantes"
<br> Usted podrá aplicar aquí otros métodos

In [16]:
if( !require("mice")) install.packages("mice", repos = "http://cran.us.r-project.org")
require("mice")

In [17]:
# Escrito por alumnos de  Universidad Austral  Rosario

Corregir_MICE <- function(pcampo, pmeses) {

  meth <- rep("", ncol(dataset))
  names(meth) <- colnames(dataset)
  meth[names(meth) == pcampo] <- "sample"

  # llamada a mice  !
  imputacion <- mice(dataset,
    method = meth,
    maxit = 5,
    m = 1,
    seed = 7)

  tbl <- mice::complete(dataset)

  dataset[, paste0(pcampo) := ifelse(foto_mes %in% pmeses, tbl[, get(pcampo)], get(pcampo))]

}


In [18]:
Corregir_interpolar <- function(pcampo, pmeses) {

  tbl <- dataset[, list(
    "v1" = shift(get(pcampo), 1, type = "lag"),
    "v2" = shift(get(pcampo), 1, type = "lead")
  ),
  by = eval(envg$PARAM$dataset_metadata$entity_id)
  ]

  tbl[, paste0(envg$PARAM$dataset_metadata$entity_id) := NULL]
  tbl[, promedio := rowMeans(tbl, na.rm = TRUE)]

  dataset[
    ,
    paste0(pcampo) := ifelse(!(foto_mes %in% pmeses),
      get(pcampo),
      tbl$promedio
    )
  ]
}

In [19]:
AsignarNA_campomeses <- function(pcampo, pmeses) {

  if( pcampo %in% colnames( dataset ) ) {

    dataset[ foto_mes %in% pmeses, paste0(pcampo) := NA ]
  }
}

In [20]:

Corregir_atributo <- function(pcampo, pmeses, pmetodo)
{
  # si el campo no existe en el dataset, Afuera !
  if( !(pcampo %in% colnames( dataset )) )
    return( 1 )

  # llamo a la funcion especializada que corresponde
  switch( pmetodo,
    "MachineLearning"     = AsignarNA_campomeses(pcampo, pmeses),
    "EstadisticaClasica"  = Corregir_interpolar(pcampo, pmeses),
    "MICE"                = Corregir_MICE(pcampo, pmeses),
  )

  return( 0 )
}

In [21]:

Corregir_Rotas <- function(dataset, pmetodo) {
  gc(verbose= FALSE)
  cat( "inicio Corregir_Rotas()\n")
  # acomodo los errores del dataset

  Corregir_atributo("active_quarter", c(202006), pmetodo) # 1
  Corregir_atributo("internet", c(202006), pmetodo) # 2

  Corregir_atributo("mrentabilidad", c(201905, 201910, 202006), pmetodo) # 3
  Corregir_atributo("mrentabilidad_annual", c(201905, 201910, 202006), pmetodo) # 4

  Corregir_atributo("mcomisiones", c(201905, 201910, 202006), pmetodo) # 5

  Corregir_atributo("mactivos_margen", c(201905, 201910, 202006), pmetodo) # 6
  Corregir_atributo("mpasivos_margen", c(201905, 201910, 202006), pmetodo) # 7

  Corregir_atributo("mcuentas_saldo", c(202006), pmetodo) # 8

  Corregir_atributo("ctarjeta_debito_transacciones", c(202006), pmetodo) # 9

  Corregir_atributo("mautoservicio", c(202006), pmetodo) # 10

  Corregir_atributo("ctarjeta_visa_transacciones", c(202006), pmetodo) # 11
  Corregir_atributo("mtarjeta_visa_consumo", c(202006), pmetodo) # 12

  Corregir_atributo("ctarjeta_master_transacciones", c(202006), pmetodo) # 13
  Corregir_atributo("mtarjeta_master_consumo", c(202006), pmetodo) # 14

  Corregir_atributo("ctarjeta_visa_debitos_automaticos", c(201904), pmetodo) # 15
  Corregir_atributo("mttarjeta_visa_debitos_automaticos", c(201904), pmetodo) # 16

  Corregir_atributo("ccajeros_propios_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 17

  Corregir_atributo("mcajeros_propios_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 18

  Corregir_atributo("ctarjeta_visa_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 19

  Corregir_atributo("mtarjeta_visa_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 20

  Corregir_atributo("ctarjeta_master_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 21

  Corregir_atributo("mtarjeta_master_descuentos",
    c(201910, 202002, 202006, 202009, 202010, 202102), pmetodo) # 22

  Corregir_atributo("ccomisiones_otras", c(201905, 201910, 202006), pmetodo) # 23
  Corregir_atributo("mcomisiones_otras", c(201905, 201910, 202006), pmetodo) # 24

  Corregir_atributo("cextraccion_autoservicio", c(202006), pmetodo) # 25
  Corregir_atributo("mextraccion_autoservicio", c(202006), pmetodo) # 26

  Corregir_atributo("ccheques_depositados", c(202006), pmetodo) # 27
  Corregir_atributo("mcheques_depositados", c(202006), pmetodo) # 28
  Corregir_atributo("ccheques_emitidos", c(202006), pmetodo) # 29
  Corregir_atributo("mcheques_emitidos", c(202006), pmetodo) # 30
  Corregir_atributo("ccheques_depositados_rechazados", c(202006), pmetodo) # 31
  Corregir_atributo("mcheques_depositados_rechazados", c(202006), pmetodo) # 32
  Corregir_atributo("ccheques_emitidos_rechazados", c(202006), pmetodo) # 33
  Corregir_atributo("mcheques_emitidos_rechazados", c(202006), pmetodo) # 34

  Corregir_atributo("tcallcenter", c(202006), pmetodo) # 35
  Corregir_atributo("ccallcenter_transacciones", c(202006), pmetodo) # 36

  Corregir_atributo("thomebanking", c(202006), pmetodo) # 37
  Corregir_atributo("chomebanking_transacciones", c(201910, 202006), pmetodo) # 38

  Corregir_atributo("ccajas_transacciones", c(202006), pmetodo) # 39
  Corregir_atributo("ccajas_consultas", c(202006), pmetodo) # 40

  Corregir_atributo("ccajas_depositos", c(202006, 202105), pmetodo) # 41

  Corregir_atributo("ccajas_extracciones", c(202006), pmetodo) # 41
  Corregir_atributo("ccajas_otras", c(202006), pmetodo) # 43

  Corregir_atributo("catm_trx", c(202006), pmetodo) # 44
  Corregir_atributo("matm", c(202006), pmetodo) # 45
  Corregir_atributo("catm_trx_other", c(202006), pmetodo) # 46
  Corregir_atributo("matm_other", c(202006), pmetodo) # 47

  cat( "fin Corregir_rotas()\n")
}


In [22]:
# resuelvo el Catastrophe Analysis

setorder( dataset, numero_de_cliente, foto_mes )

PARAM$CA$metodo= "MachineLearning"

if( PARAM$CA$metodo %in% c("MachineLearning", "EstadisticaClasica", "MICE") )
  Corregir_Rotas(dataset, PARAM$CA$metodo)

inicio Corregir_Rotas()
fin Corregir_rotas()


#### 9.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, ajustando por algunos indices financieros

In [23]:
# meses que me interesan para el ajuste de variables monetarias
vfoto_mes <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106,
  202107, 202108, 202109
)


In [24]:
# los valores que siguen fueron calculados por alumnos

# momento 1.0  31-dic-2020 a las 23:59
vIPC <- c(
  1.9903030878, 1.9174403544, 1.8296186587,
  1.7728862972, 1.7212488323, 1.6776304408,
  1.6431248196, 1.5814483345, 1.4947526791,
  1.4484037589, 1.3913580777, 1.3404220402,
  1.3154288912, 1.2921698342, 1.2472681797,
  1.2300475145, 1.2118694724, 1.1881073259,
  1.1693969743, 1.1375456949, 1.1065619600,
  1.0681100000, 1.0370000000, 1.0000000000,
  0.9680542110, 0.9344152616, 0.8882274350,
  0.8532444140, 0.8251880213, 0.8003763543,
  0.7763107219, 0.7566381305, 0.7289384687
)

vdolar_blue <- c(
   39.045455,  38.402500,  41.639474,
   44.274737,  46.095455,  45.063333,
   43.983333,  54.842857,  61.059524,
   65.545455,  66.750000,  72.368421,
   77.477273,  78.191667,  82.434211,
  101.087500, 126.236842, 125.857143,
  130.782609, 133.400000, 137.954545,
  170.619048, 160.400000, 153.052632,
  157.900000, 149.380952, 143.615385,
  146.250000, 153.550000, 162.000000,
  178.478261, 180.878788, 184.357143
)

vdolar_oficial <- c(
   38.430000,  39.428000,  42.542105,
   44.354211,  46.088636,  44.955000,
   43.751429,  54.650476,  58.790000,
   61.403182,  63.012632,  63.011579,
   62.983636,  63.580556,  65.200000,
   67.872000,  70.047895,  72.520952,
   75.324286,  77.488500,  79.430909,
   83.134762,  85.484737,  88.181667,
   91.474000,  93.997778,  96.635909,
   98.526000,  99.613158, 100.619048,
  101.619048, 102.569048, 103.781818
)

vUVA <- c(
  2.001408838932958,  1.950325472789153,  1.89323032351521,
  1.8247220405493787, 1.746027787673673,  1.6871348409529485,
  1.6361678865622313, 1.5927529755859773, 1.5549162794128493,
  1.4949100586391746, 1.4197729500774545, 1.3678188186372326,
  1.3136508617223726, 1.2690535173062818, 1.2381595983200178,
  1.211656735577568,  1.1770808941405335, 1.1570338657445522,
  1.1388769475653255, 1.1156993751209352, 1.093638313080772,
  1.0657171590878205, 1.0362173587708712, 1.0,
  0.9669867858358365, 0.9323750098728378, 0.8958202912590305,
  0.8631993702994263, 0.8253893405524657, 0.7928918905364516,
  0.7666323845128089, 0.7428976357662823, 0.721615762047849
)


In [25]:
tb_indices <- as.data.table( list(
  "IPC" = vIPC,
  "dolar_blue" = vdolar_blue,
  "dolar_oficial" = vdolar_oficial,
  "UVA" = vUVA
  )
)

tb_indices[[ 'foto_mes' ]] <- vfoto_mes

tb_indices

IPC,dolar_blue,dolar_oficial,UVA,foto_mes
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1.9903031,39.04545,38.43000,2.0014088,201901
1.9174404,38.40250,39.42800,1.9503255,201902
1.8296187,41.63947,42.54210,1.8932303,201903
1.7728863,44.27474,44.35421,1.8247220,201904
1.7212488,46.09546,46.08864,1.7460278,201905
1.6776304,45.06333,44.95500,1.6871348,201906
1.6431248,43.98333,43.75143,1.6361679,201907
1.5814483,54.84286,54.65048,1.5927530,201908
1.4947527,61.05952,58.79000,1.5549163,201909


In [26]:
drift_UVA <- function(campos_monetarios) {
  cat( "inicio drift_UVA()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD * i.UVA,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_UVA()\n")
}


In [27]:
drift_dolar_oficial <- function(campos_monetarios) {
  cat( "inicio drift_dolar_oficial()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD / i.dolar_oficial,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_dolar_oficial()\n")
}


In [28]:
drift_dolar_blue <- function(campos_monetarios) {
  cat( "inicio drift_dolar_blue()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD / i.dolar_blue,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_dolar_blue()\n")
}


In [29]:
drift_deflacion <- function(campos_monetarios) {
  cat( "inicio drift_deflacion()\n")

  dataset[tb_indices,
    on = c("foto_mes"),
    (campos_monetarios) := .SD * i.IPC,
    .SDcols = campos_monetarios
  ]

  cat( "fin drift_deflacion()\n")
}


In [30]:
drift_rank_simple <- function(campos_drift) {

  cat( "inicio drift_rank_simple()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[, paste0(campo, "_rank") :=
      (frank(get(campo), ties.method = "random") - 1) / (.N - 1), by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat( "fin drift_rank_simple()\n")
}


In [31]:
# El cero se transforma en cero
# los positivos se rankean por su lado
# los negativos se rankean por su lado

drift_rank_cero_fijo <- function(campos_drift) {

  cat( "inicio drift_rank_cero_fijo()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[get(campo) == 0, paste0(campo, "_rank") := 0]
    dataset[get(campo) > 0, paste0(campo, "_rank") :=
      frank(get(campo), ties.method = "random") / .N, by = list(foto_mes)]

    dataset[get(campo) < 0, paste0(campo, "_rank") :=
      -frank(-get(campo), ties.method = "random") / .N, by = list(foto_mes)]
    dataset[, (campo) := NULL]
  }
  cat("\n")
  cat( "fin drift_rank_cero_fijo()\n")
}


In [32]:
drift_estandarizar <- function(campos_drift) {

  cat( "inicio drift_estandarizar()\n")
  for (campo in campos_drift)
  {
    cat(campo, " ")
    dataset[, paste0(campo, "_normal") :=
      (get(campo) -mean(campo, na.rm=TRUE)) / sd(get(campo), na.rm=TRUE),
      by = list(foto_mes)]

    dataset[, (campo) := NULL]
  }
  cat( "fin drift_estandarizar()\n")
}


In [33]:
# por como armé los nombres de campos,
#  estos son los campos que expresan variables monetarias
campos_monetarios <- colnames(dataset)
campos_monetarios <- campos_monetarios[campos_monetarios %like%
  "^(m|Visa_m|Master_m|vm_m)"]

campos_monetarios

[1] "mrentabilidad"                      "mrentabilidad_annual"              
 [3] "mcomisiones"                        "mactivos_margen"                   
 [5] "mpasivos_margen"                    "mcuenta_corriente"                 
 [7] "mcaja_ahorro"                       "mcuentas_saldo"                    
 [9] "mtarjeta_visa_consumo"              "mtarjeta_master_consumo"           
[11] "mprestamos_personales"              "mpayroll"                          
[13] "mttarjeta_visa_debitos_automaticos" "mcomisiones_mantenimiento"         
[15] "mtransferencias_recibidas"          "Master_mfinanciacion_limite"       
[17] "Master_msaldototal"                 "Master_mlimitecompra"              
[19] "Master_mconsumototal"               "Master_mpagominimo"                
[21] "Visa_mfinanciacion_limite"          "Visa_msaldototal"                  
[23] "Visa_mlimitecompra"                 "Visa_mconsumototal"                
[25] "Visa_mpagominimo"

In [34]:
# ejecuto el Data Drifting
setorder( dataset, numero_de_cliente, foto_mes )


PARAM$DR$metodo <- "deflacion"

switch(PARAM$DR$metodo,
  "ninguno"        = cat("No hay correccion del data drifting"),
  "rank_simple"    = drift_rank_simple(campos_monetarios),
  "rank_cero_fijo" = drift_rank_cero_fijo(campos_monetarios),
  "deflacion"      = drift_deflacion(campos_monetarios),
  "dolar_blue"     = drift_dolarblue(campos_monetarios),
  "dolar_oficial"  = drift_dolaroficial(campos_monetarios),
  "UVA"            = drift_UVA(campos_monetarios),
  "estandarizar"   = drift_estandarizar(campos_monetarios)
)


inicio drift_deflacion()
fin drift_deflacion()


In [35]:
colnames(dataset)

[1] "numero_de_cliente"                  "foto_mes"                          
 [3] "internet"                           "cliente_edad"                      
 [5] "cliente_antiguedad"                 "mrentabilidad"                     
 [7] "mrentabilidad_annual"               "mcomisiones"                       
 [9] "mactivos_margen"                    "mpasivos_margen"                   
[11] "cproductos"                         "mcuenta_corriente"                 
[13] "mcaja_ahorro"                       "cdescubierto_preacordado"          
[15] "mcuentas_saldo"                     "ctarjeta_visa"                     
[17] "ctarjeta_visa_transacciones"        "mtarjeta_visa_consumo"             
[19] "ctarjeta_master"                    "ctarjeta_master_transacciones"     
[21] "mtarjeta_master_consumo"            "cprestamos_personales"             
[23] "mprestamos_personales"              "cpayroll_trx"                      
[25] "mpayroll"                           "mttarjeta_visa_debitos_automaticos"
[27] "ccomisiones_mantenimiento"          "mcomisiones_mantenimiento"         
[29] "ccomisiones_otras"                  "mtransferencias_recibidas"         
[31] "ccallcenter_transacciones"          "thomebanking"                      
[33] "chomebanking_transacciones"         "ctrx_quarter"                      
[35] "Master_status"                      "Master_mfinanciacion_limite"       
[37] "Master_Fvencimiento"                "Master_msaldototal"                
[39] "Master_mlimitecompra"               "Master_fultimo_cierre"             
[41] "Master_fechaalta"                   "Master_mconsumototal"              
[43] "Master_cconsumos"                   "Master_mpagominimo"                
[45] "Visa_status"                        "Visa_mfinanciacion_limite"         
[47] "Visa_Fvencimiento"                  "Visa_msaldototal"                  
[49] "Visa_mlimitecompra"                 "Visa_fultimo_cierre"               
[51] "Visa_fechaalta"                     "Visa_mconsumototal"                
[53] "Visa_cconsumos"                     "Visa_mpagominimo"                  
[55] "clase_ternaria"

In [36]:
# se intenta corregir el data drifting utilizando algunos indices financieros

#### 9.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [37]:
# esta funcion atributos presentes existe debido a que las modalidades poseen datasets con distinta cantidad de campos
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}

# el mes 1,2, ..12
if( atributos_presentes( c("foto_mes") ))
  dataset[, kmes := foto_mes %% 100]

# variable extraida de una tesis de maestria de Irlanda
if( atributos_presentes( c("mpayroll", "cliente_edad") ))
  dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

# Evitar divisiones por cero
safe_div <- function(a, b) {
  return( ifelse(b == 0 | is.na(b), NA, a / b) )
}


In [38]:

# Ratios nuevos
dataset[, uso_limite_visa       := safe_div(Visa_msaldototal, Visa_mlimitecompra)]
dataset[, uso_limite_master     := safe_div(Master_msaldototal, Master_mlimitecompra)]
dataset[, consumo_vs_limite_visa   := safe_div(mtarjeta_visa_consumo, Visa_mlimitecompra)]
dataset[, consumo_vs_limite_master := safe_div(mtarjeta_master_consumo, Master_mlimitecompra)]
dataset[, ratio_pago_min_visa   := safe_div(Visa_mpagominimo, Visa_msaldototal)]
dataset[, ratio_pago_min_master := safe_div(Master_mpagominimo, Master_msaldototal)]
dataset[, hb_ratio              := safe_div(chomebanking_transacciones, thomebanking)]
dataset[, call_ratio            := safe_div(ccallcenter_transacciones, cproductos)]
dataset[, consumo_tarjetas      := mtarjeta_visa_consumo + mtarjeta_master_consumo]
dataset[, uso_total_tarjetas    := safe_div(consumo_tarjetas, (Master_mlimitecompra + Visa_mlimitecompra))]


In [39]:
# visualizo las columas del dataset a esta etapa
colnames(dataset)

[1] "numero_de_cliente"                  "foto_mes"                          
 [3] "internet"                           "cliente_edad"                      
 [5] "cliente_antiguedad"                 "mrentabilidad"                     
 [7] "mrentabilidad_annual"               "mcomisiones"                       
 [9] "mactivos_margen"                    "mpasivos_margen"                   
[11] "cproductos"                         "mcuenta_corriente"                 
[13] "mcaja_ahorro"                       "cdescubierto_preacordado"          
[15] "mcuentas_saldo"                     "ctarjeta_visa"                     
[17] "ctarjeta_visa_transacciones"        "mtarjeta_visa_consumo"             
[19] "ctarjeta_master"                    "ctarjeta_master_transacciones"     
[21] "mtarjeta_master_consumo"            "cprestamos_personales"             
[23] "mprestamos_personales"              "cpayroll_trx"                      
[25] "mpayroll"                           "mttarjeta_visa_debitos_automaticos"
[27] "ccomisiones_mantenimiento"          "mcomisiones_mantenimiento"         
[29] "ccomisiones_otras"                  "mtransferencias_recibidas"         
[31] "ccallcenter_transacciones"          "thomebanking"                      
[33] "chomebanking_transacciones"         "ctrx_quarter"                      
[35] "Master_status"                      "Master_mfinanciacion_limite"       
[37] "Master_Fvencimiento"                "Master_msaldototal"                
[39] "Master_mlimitecompra"               "Master_fultimo_cierre"             
[41] "Master_fechaalta"                   "Master_mconsumototal"              
[43] "Master_cconsumos"                   "Master_mpagominimo"                
[45] "Visa_status"                        "Visa_mfinanciacion_limite"         
[47] "Visa_Fvencimiento"                  "Visa_msaldototal"                  
[49] "Visa_mlimitecompra"                 "Visa_fultimo_cierre"               
[51] "Visa_fechaalta"                     "Visa_mconsumototal"                
[53] "Visa_cconsumos"                     "Visa_mpagominimo"                  
[55] "clase_ternaria"                     "kmes"                              
[57] "mpayroll_sobre_edad"                "uso_limite_visa"                   
[59] "uso_limite_master"                  "consumo_vs_limite_visa"            
[61] "consumo_vs_limite_master"           "ratio_pago_min_visa"               
[63] "ratio_pago_min_master"              "hb_ratio"                          
[65] "call_ratio"                         "consumo_tarjetas"                  
[67] "uso_total_tarjetas"

#### 9.3.1.4  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest

In [40]:
if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

Loading required package: lightgbm



In [41]:
AgregaVarRandomForest <- function() {

  cat( "inicio AgregaVarRandomForest()\n")
  gc(verbose= FALSE)
  dataset[, clase01 := 0L ]
  dataset[ clase_ternaria %in% PARAM$FE_rf$train$clase01_valor1,
      clase01 := 1L ]

  campos_buenos <- setdiff(
    colnames(dataset),
    c( "clase_ternaria", "clase01")
  )

  dataset[, entrenamiento :=
    as.integer( foto_mes %in% PARAM$FE_rf$train$training )]

  dtrain <- lgb.Dataset(
    data = data.matrix(dataset[entrenamiento == TRUE, campos_buenos, with = FALSE]),
    label = dataset[entrenamiento == TRUE, clase01],
    free_raw_data = FALSE
  )

  modelo <- lgb.train(
     data = dtrain,
     param = PARAM$FE_rf$lgb_param,
     verbose = -100
  )

  cat( "Fin construccion RandomForest\n" )
  # grabo el modelo, achivo .model
  lgb.save(modelo, file="modelo.model" )

  qarbolitos <- copy(PARAM$FE_rf$lgb_param$num_iterations)

  periodos <- dataset[ , unique( foto_mes ) ]

  for( periodo in  periodos )
  {
    cat( "periodo = ", periodo, "\n" )
    datamatrix <- data.matrix(dataset[ foto_mes== periodo, campos_buenos, with = FALSE])

    cat( "Inicio prediccion\n" )
    prediccion <- predict(
        modelo,
        datamatrix,
        type = "leaf"
    )
    cat( "Fin prediccion\n" )

    for( arbolito in 1:qarbolitos )
    {
       cat( arbolito, " " )
       hojas_arbol <- unique(prediccion[ , arbolito])

       for (pos in 1:length(hojas_arbol)) {
         # el numero de nodo de la hoja, estan salteados
         nodo_id <- hojas_arbol[pos]
         dataset[ foto_mes== periodo, paste0(
            "rf_", sprintf("%03d", arbolito),
             "_", sprintf("%03d", nodo_id)
          ) :=  as.integer( nodo_id == prediccion[ , arbolito]) ]

       }

       rm( hojas_arbol )
    }
    cat( "\n" )

    rm( prediccion )
    rm( datamatrix )
    gc(verbose= FALSE)
  }

  gc(verbose= FALSE)

  # borro clase01 , no debe ensuciar el dataset
  dataset[ , clase01 := NULL ]

}


In [42]:
# Parametros de Feature Engineering  a partir de hojas de Random Forest

# Estos CUATRO parametros son los que se deben modificar
PARAM$FE_rf$arbolitos= 20
PARAM$FE_rf$hojas_por_arbol= 16
PARAM$FE_rf$datos_por_hoja= 100
PARAM$FE_rf$mtry_ratio= 0.2

# Estos son quasi fijos
PARAM$FE_rf$train$clase01_valor1 <- c( "BAJA+2", "BAJA+1")
PARAM$FE_rf$train$training <- c( 202101, 202102, 202103)

# Estos TAMBIEN son quasi fijos
PARAM$FE_rf$lgb_param <-list(
    # parametros que se pueden cambiar
    num_iterations = PARAM$FE_rf$arbolitos,
    num_leaves  = PARAM$FE_rf$hojas_por_arbol,
    min_data_in_leaf = PARAM$FE_rf$datos_por_hoja,
    feature_fraction_bynode  = PARAM$FE_rf$mtry_ratio,

    # para que LightGBM emule Random Forest
    boosting = "rf",
    bagging_fraction = ( 1.0 - 1.0/exp(1.0) ),
    bagging_freq = 1.0,
    feature_fraction = 1.0,

    # genericos de LightGBM
    max_bin = 31L,
    objective = "binary",
    first_metric_only = TRUE,
    boost_from_average = TRUE,
    feature_pre_filter = FALSE,
    force_row_wise = TRUE,
    verbosity = -100,
    max_depth = -1L,
    min_gain_to_split = 0.0,
    min_sum_hessian_in_leaf = 0.001,
    lambda_l1 = 0.0,
    lambda_l2 = 0.0,

    pos_bagging_fraction = 1.0,
    neg_bagging_fraction = 1.0,
    is_unbalance = FALSE,
    scale_pos_weight = 1.0,

    drop_rate = 0.1,
    max_drop = 50,
    skip_drop = 0.5,

    extra_trees = FALSE
  )

In [43]:
# Feature Engineering agregando variables de Random Forest
AgregaVarRandomForest()

inicio AgregaVarRandomForest()
Fin construccion RandomForest
periodo =  202003 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202004 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202005 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202006 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202007 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202008 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202009 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7  8  9  10  11  12  13  14  15  16  17  18  19  20  
periodo =  202010 
Inicio prediccion
Fin prediccion
1  2  3  4  5  6  7

In [44]:
ncol(dataset)
colnames(dataset)

[1] 388

[1] "numero_de_cliente"                  "foto_mes"                          
  [3] "internet"                           "cliente_edad"                      
  [5] "cliente_antiguedad"                 "mrentabilidad"                     
  [7] "mrentabilidad_annual"               "mcomisiones"                       
  [9] "mactivos_margen"                    "mpasivos_margen"                   
 [11] "cproductos"                         "mcuenta_corriente"                 
 [13] "mcaja_ahorro"                       "cdescubierto_preacordado"          
 [15] "mcuentas_saldo"                     "ctarjeta_visa"                     
 [17] "ctarjeta_visa_transacciones"        "mtarjeta_visa_consumo"             
 [19] "ctarjeta_master"                    "ctarjeta_master_transacciones"     
 [21] "mtarjeta_master_consumo"            "cprestamos_personales"             
 [23] "mprestamos_personales"              "cpayroll_trx"                      
 [25] "mpayroll"                           "mttarjeta_visa_debitos_automaticos"
 [27] "ccomisiones_mantenimiento"          "mcomisiones_mantenimiento"         
 [29] "ccomisiones_otras"                  "mtransferencias_recibidas"         
 [31] "ccallcenter_transacciones"          "thomebanking"                      
 [33] "chomebanking_transacciones"         "ctrx_quarter"                      
 [35] "Master_status"                      "Master_mfinanciacion_limite"       
 [37] "Master_Fvencimiento"                "Master_msaldototal"                
 [39] "Master_mlimitecompra"               "Master_fultimo_cierre"             
 [41] "Master_fechaalta"                   "Master_mconsumototal"              
 [43] "Master_cconsumos"                   "Master_mpagominimo"                
 [45] "Visa_status"                        "Visa_mfinanciacion_limite"         
 [47] "Visa_Fvencimiento"                  "Visa_msaldototal"                  
 [49] "Visa_mlimitecompra"                 "Visa_fultimo_cierre"               
 [51] "Visa_fechaalta"                     "Visa_mconsumototal"                
 [53] "Visa_cconsumos"                     "Visa_mpagominimo"                  
 [55] "clase_ternaria"                     "kmes"                              
 [57] "mpayroll_sobre_edad"                "uso_limite_visa"                   
 [59] "uso_limite_master"                  "consumo_vs_limite_visa"            
 [61] "consumo_vs_limite_master"           "ratio_pago_min_visa"               
 [63] "ratio_pago_min_master"              "hb_ratio"                          
 [65] "call_ratio"                         "consumo_tarjetas"                  
 [67] "uso_total_tarjetas"                 "entrenamiento"                     
 [69] "rf_001_009"                         "rf_001_010"                        
 [71] "rf_001_014"                         "rf_001_006"                        
 [73] "rf_001_002"                         "rf_001_001"                        
 [75] "rf_001_008"                         "rf_001_004"                        
 [77] "rf_001_011"                         "rf_001_013"                        
 [79] "rf_001_015"                         "rf_001_005"                        
 [81] "rf_001_000"                         "rf_001_012"                        
 [83] "rf_001_007"                         "rf_001_003"                        
 [85] "rf_002_009"                         "rf_002_014"                        
 [87] "rf_002_002"                         "rf_002_003"                        
 [89] "rf_002_001"                         "rf_002_011"                        
 [91] "rf_002_010"                         "rf_002_004"                        
 [93] "rf_002_005"                         "rf_002_015"                        
 [95] "rf_002_008"                         "rf_002_012"                        
 [97] "rf_002_013"                         "rf_002_000"                        
 [99] "rf_002_007"                         "rf_002_006"                        
[1

In [45]:
# No se implementa Feature Engineering a partir de Random Forest

#### 9.3.1.5  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

In [46]:
# Feature Engineering Historico

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


Verificacion de los campos recien creados

In [47]:
ncol(dataset)
colnames(dataset)

[1] 1928

[1] "numero_de_cliente"                        
   [2] "foto_mes"                                 
   [3] "internet"                                 
   [4] "cliente_edad"                             
   [5] "cliente_antiguedad"                       
   [6] "mrentabilidad"                            
   [7] "mrentabilidad_annual"                     
   [8] "mcomisiones"                              
   [9] "mactivos_margen"                          
  [10] "mpasivos_margen"                          
  [11] "cproductos"                               
  [12] "mcuenta_corriente"                        
  [13] "mcaja_ahorro"                             
  [14] "cdescubierto_preacordado"                 
  [15] "mcuentas_saldo"                           
  [16] "ctarjeta_visa"                            
  [17] "ctarjeta_visa_transacciones"              
  [18] "mtarjeta_visa_consumo"                    
  [19] "ctarjeta_master"                          
  [20] "ctarjeta_master_transacciones"            
  [21] "mtarjeta_master_consumo"                  
  [22] "cprestamos_personales"                    
  [23] "mprestamos_personales"                    
  [24] "cpayroll_trx"                             
  [25] "mpayroll"                                 
  [26] "mttarjeta_visa_debitos_automaticos"       
  [27] "ccomisiones_mantenimiento"                
  [28] "mcomisiones_mantenimiento"                
  [29] "ccomisiones_otras"                        
  [30] "mtransferencias_recibidas"                
  [31] "ccallcenter_transacciones"                
  [32] "thomebanking"                             
  [33] "chomebanking_transacciones"               
  [34] "ctrx_quarter"                             
  [35] "Master_status"                            
  [36] "Master_mfinanciacion_limite"              
  [37] "Master_Fvencimiento"                      
  [38] "Master_msaldototal"                       
  [39] "Master_mlimitecompra"                     
  [40] "Master_fultimo_cierre"                    
  [41] "Master_fechaalta"                         
  [42] "Master_mconsumototal"                     
  [43] "Master_cconsumos"                         
  [44] "Master_mpagominimo"                       
  [45] "Visa_status"                              
  [46] "Visa_mfinanciacion_limite"                
  [47] "Visa_Fvencimiento"                        
  [48] "Visa_msaldototal"                         
  [49] "Visa_mlimitecompra"                       
  [50] "Visa_fultimo_cierre"                      
  [51] "Visa_fechaalta"                           
  [52] "Visa_mconsumototal"                       
  [53] "Visa_cconsumos"                           
  [54] "Visa_mpagominimo"                         
  [55] "clase_ternaria"                           
  [56] "kmes"                                     
  [57] "mpayroll_sobre_edad"                      
  [58] "uso_limite_visa"                          
  [59] "uso_limite_master"                        
  [60] "consumo_vs_limite_visa"                   
  [61] "consumo_vs_limite_master"                 
  [62] "ratio_pago_min_visa"                      
  [63] "ratio_pago_min_master"                    
  [64] "hb_ratio"                                 
  [65] "call_ratio"                               
  [66] "consumo_tarjetas"                         
  [67] "uso_total_tarjetas"                       
  [68] "entrenamiento"                            
  [69] "rf_001_009"                               
  [70] "rf_001_010"                               
  [71] "rf_001_014"                               
  [72] "rf_001_006"                               
  [73] "rf_001_002"                               
  [74] "rf_001_001"                               
  [75] "rf_001_008"                               
  [76] "rf_001_004"                               
  [77] "rf_001_011"                               
  [78] "rf_001_013"                               
  [79] "rf_001_015"      

#### 9.3.1.6  FEhist Reduccion dimensionalidad con canaritos

Esta etapa solo se mostrará a la *modalidad Anlista Sr* por algun canal secreto de forma de no confundir a los *Analista Jr*  nni distraer con detalles operativos a la estratégica *Modalidad Gerencial*

In [48]:
# No se implementa la reduccion de la dimensionalidad con canaritos

### 9.3.2 Modelado

#### 9.3.2.1 Training Strategy

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 201901, 202107 ]  SIN undersampling

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling
   * training = [ 201901, 202105 ]  donde se consideran el 100% de los CONTINUA

In [49]:
PARAM$trainingstrategy$validate <- c(202107)

PARAM$trainingstrategy$training <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105
)


PARAM$trainingstrategy$training_pct <- 1.0


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [50]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [51]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [52]:
# preparo para que se puede hacer undersampling de los CONTINUA
#  solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]


if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  free_raw_data= TRUE
)

In [53]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

[1] 33053

####  9.3.2.2. Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [54]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers


Attaching package: ‘ParamHelpers’


The following object is masked from ‘package:R.utils’:

    isVector



Attaching package: ‘mlr’


The following objects are masked from ‘package:R.utils’:

    resample, setThreshold


Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames


The following object is masked from ‘package:R.utils’:

    asInt



Attaching package: ‘smoof’


The following objects are masked from ‘package:R.oo’:

    getDescription, getName




Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [55]:
# un Analista Jr  debe poder animarse a hacer 100 iteraciones
PARAM$hipeparametertuning$num_interations <- 50

# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  max_bin= 31,

  num_iterations= 9999,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 400
)

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeIntegerParam("num_leaves", lower= 2L, upper= 256L),
  makeIntegerParam("min_data_in_leaf", lower= 2L, upper= 8192L),
  makeNumericParam("feature_fraction", lower= 0.4, upper= 0.9),
  makeNumericParam("learning_rate", lower= 0.01, upper= 0.5)
)

Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

In [56]:
# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

In [57]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize= FALSE, # estoy Maximizando AUC
    noisy= FALSE,
    par.set= PARAM$hipeparametertuning$hs,
    has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= "HT.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace = TRUE)
)

Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

In [58]:
# inicio la optimizacion bayesiana

if (!file.exists("HT.RDATA")) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue("HT.RDATA") # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.

Sat Nov 29 20:29:00 2025 AUC 0.922916051684034

Sat Nov 29 20:31:28 2025 AUC 0.925339857569427

Sat Nov 29 20:34:10 2025 AUC 0.919574447504368

Sat Nov 29 20:37:31 2025 AUC 0.923818006357938

Sat Nov 29 20:47:25 2025 AUC 0.904706013880917

Sat Nov 29 20:49:52 2025 AUC 0.915380772229969

Sat Nov 29 20:53:42 2025 AUC 0.906265091644027

Sat Nov 29 20:59:25 2025 AUC 0.910910190973997

Sat Nov 29 21:03:52 2025 AUC 0.925350936191189

Sat Nov 29 21:09:51 2025 AUC 0.927601147221065

Sat Nov 29 21:12:55 2025 AUC 0.927567613543365

Sat Nov 29 21:16:01 2025 AUC 0.893529084241125

Sat Nov 29 21:26:36 2025 AUC 0.887233865893403

Sat Nov 29 21:41:04 2025 AUC 0.930089905993725

Sat Nov 29 21:47:13 2025 AUC 0.924294893374817

Sat Nov 29 21:50:17 2025 AUC 0.924877325110852

[mbo] 0: num_leaves=167; min_data_in_leaf=4296; feature_fraction=0.609; learning_rate=0.242 : y = 0.923 : 253.9 secs : initdesign

[mbo] 0: num_leaves=103; min_data_in_leaf=2594; feat

la bayesian optimization ha corrido, extraigo los mejores hiperparametros

In [59]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y, -num_iterations)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file="BO_log.txt",
  sep="\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]

print(PARAM$out$lgbm$mejores_hiperparametros)

   num_leaves min_data_in_leaf feature_fraction learning_rate num_iterations
        <int>            <int>            <num>         <num>          <int>
1:         87             1758         0.620652    0.01018209           1060


### 9.3.3 Produccion

#### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

##### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [60]:
PARAM$trainingstrategy$final_train <- c(
  201901, 201902, 201903, 201904, 201905, 201906,
  201907, 201908, 201909, 201910, 201911, 201912,
  202001, 202002, 202003, 202004, 202005, 202006,
  202007, 202008, 202009, 202010, 202011, 202012,
  202101, 202102, 202103, 202104, 202105, 202106,
  202107
)


dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

# creo el dfinal_train en formato  LightGBM
dfinal_train <- lgb.Dataset(
  data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
  label= dataset[fold_final_train == TRUE, clase01],
  free_raw_data= TRUE
)

nrow( dfinal_train) # verifico el tamaño

[1] 910211

##### Final Training Hyperparameters

In [61]:
# uno los parametros fijos y los mejores encontrados de los variables
fijos <- copy(PARAM$lgbm$param_fijos)

# quito lo que optimice en la Bayesian Optimization
fijos$num_iterations <- NULL
fijos$early_stopping_rounds <- NULL

# agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

##### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria

# SEMILLERO AÑADIDO
## Cambio todo hasta scoring

In [62]:
PARAM$FT$semillerio <- 50  # cantidad de semillas

In [63]:
if(!require("primes")) install.packages("primes")
require("primes")

Loading required package: primes



In [64]:
primos <- generate_primes(min = 100000, max = 1000000)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
# me quedo con PARAM$semillerio  primos al azar
PARAM$FT$semillas <- sample(primos)[seq(PARAM$FT$semillerio)]

cat( PARAM$FT$semillas)

599719 251887 955217 179939 314927 784313 918361 421703 952691 254879 608903 968197 658051 208997 792119 105863 310049 393073 425851 855737 608029 737497 163987 373487 986617 203909 427619 792223 911459 304763 175361 147211 924037 561439 237851 915641 844141 688543 615709 767051 427967 260263 537127 516233 892657 805531 175061 491137 981527 957109

In [65]:
dir.create("modelos", showWarnings =FALSE)

primero <- TRUE
for( sem in PARAM$FT$semillas)
{
  nombre_arch <- paste0( "./modelos/modelo_", sem, ".txt")
  if( !file.exists(nombre_arch) )
  {
    param_final$seed <- sem

    set.seed(sem, kind = "L'Ecuyer-CMRG")
    final_model <- lgb.train(
      data= dfinal_train,
      param= param_final,
      verbose= -100
    )

    lgb.save(final_model, nombre_arch) # grabo el modelo"

    # grabo la primer importancia de variables
    #  Natalia : da lo mismo cual se guarda
    if( primero)
    {
      primero <- FALSE
      tb_importancia <- as.data.table(lgb.importance(final_model))
      archivo_importancia <- "impo.txt"

      fwrite( tb_importancia,
        file= archivo_importancia,
        sep= "\t"
      )
    }
 }
}

#### Scoring

Aplico el modelo final a los datos del futuro

In [66]:
PARAM$trainingstrategy$future <- c(202109)

dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]

## Cambio para semillero

In [67]:
# aplico final_model   a dfuture

tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := 0]

datos_matrix <- data.matrix(dfuture[, campos_buenos, with= FALSE])


for( isem in seq(length(PARAM$FT$semillas)) )
{
   sem <- PARAM$FT$semillas[ isem ]
   nombre_arch <- paste0( "./modelos/modelo_", sem, ".txt")
   final_model <- lgb.load(nombre_arch)

   prediccion <- predict(
     final_model,
     datos_matrix
  )

  tb_prediccion[, paste0("prob_", isem) := prediccion]
  tb_prediccion[, prob := prob + prediccion]

  rm(final_model)
  rm(prediccion)
  gc(full = TRUE, verbose=FALSE)
}

rm( datos_matrix)
gc(full = TRUE, verbose=FALSE)

tb_prediccion[, prob := prob /length(PARAM$FT$semillas) ]

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,2946964,157.4,9327715,498.2,9327715,498.2
Vcells,1095736067,8359.9,3624806919,27655.1,4530957783,34568.5


In [68]:
# veo que hay en tb_prediccion
tb_prediccion

numero_de_cliente,prob,prob_1,prob_2,prob_3,prob_4,prob_5,prob_6,prob_7,prob_8,⋯,prob_41,prob_42,prob_43,prob_44,prob_45,prob_46,prob_47,prob_48,prob_49,prob_50
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
29186441,2.093367e-03,2.640467e-03,2.432382e-03,1.794362e-03,1.927221e-03,1.938078e-03,2.042044e-03,1.959815e-03,2.009119e-03,⋯,2.631105e-03,1.973190e-03,2.594489e-03,2.400437e-03,1.878932e-03,2.242761e-03,1.916379e-03,1.987803e-03,2.403589e-03,1.733026e-03
29187730,1.857263e-04,1.662721e-04,1.631334e-04,1.506294e-04,2.159751e-04,1.281467e-04,2.497115e-04,1.893977e-04,1.822499e-04,⋯,1.531151e-04,2.129208e-04,1.712189e-04,1.789675e-04,1.779218e-04,1.792005e-04,2.113560e-04,2.031882e-04,1.827115e-04,1.513186e-04
29187961,1.973600e-03,1.923609e-03,1.742264e-03,2.064383e-03,1.863607e-03,2.091723e-03,1.871020e-03,1.681542e-03,1.732735e-03,⋯,2.044253e-03,2.495595e-03,3.072976e-03,1.478320e-03,1.387213e-03,1.567848e-03,2.492702e-03,3.091546e-03,2.802307e-03,2.453122e-03
29193101,3.655218e-04,3.557688e-04,4.355988e-04,3.562075e-04,3.018778e-04,4.051685e-04,3.531092e-04,3.672120e-04,5.061180e-04,⋯,2.949343e-04,3.403288e-04,3.932372e-04,4.207899e-04,3.162282e-04,3.847573e-04,3.543360e-04,3.340525e-04,3.144217e-04,3.601495e-04
29193281,7.479235e-04,9.041562e-04,7.177714e-04,8.567453e-04,9.500804e-04,6.953516e-04,7.232629e-04,8.257701e-04,7.180455e-04,⋯,1.114453e-03,6.107666e-04,4.824835e-04,5.931787e-04,7.424268e-04,8.977038e-04,7.839460e-04,7.427664e-04,7.579659e-04,1.003959e-03
29198891,6.723277e-04,6.127093e-04,6.206577e-04,7.533612e-04,6.732604e-04,7.461500e-04,6.373557e-04,6.087612e-04,5.992987e-04,⋯,6.899102e-04,6.513676e-04,6.031914e-04,7.296613e-04,6.336385e-04,6.052215e-04,6.191043e-04,6.561119e-04,7.041566e-04,7.482637e-04
29200651,1.756474e-02,1.485686e-02,1.731863e-02,1.455857e-02,2.329647e-02,1.831764e-02,2.238973e-02,1.771808e-02,1.507348e-02,⋯,1.342961e-02,1.975780e-02,1.631103e-02,1.935066e-02,1.719367e-02,1.595756e-02,1.599018e-02,1.540104e-02,1.720312e-02,1.610289e-02
29200770,8.042197e-05,6.976154e-05,7.513990e-05,6.881933e-05,6.440875e-05,8.421628e-05,8.901589e-05,5.850354e-05,1.080028e-04,⋯,7.575212e-05,7.848078e-05,7.410008e-05,7.675224e-05,9.267511e-05,6.698272e-05,9.317878e-05,8.417219e-05,9.267403e-05,8.917991e-05
29201701,4.387575e-04,6.051511e-04,4.086662e-04,4.327605e-04,3.530560e-04,3.863255e-04,4.908505e-04,3.849789e-04,4.384473e-04,⋯,4.340806e-04,4.894747e-04,3.969606e-04,4.563228e-04,4.717957e-04,4.006830e-04,4.406611e-04,5.296465e-04,3.537952e-04,3.777588e-04


In [69]:
# grabo las probabilidad del modelo
#  me va a ser util para hacer Ensembles de modelos
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

##### Tabla Prediccion

#### Kaggle Competition Submit

Genero las salidas y hago los submits a Kaggle

In [70]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

PARAM$kaggle$competencia <- "labol-2025-virtual-analista-jr"
PARAM$kaggle$cortes <- seq(1800, 2400, by = 100)

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marclo los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  # subida a Kaggle, armo la linea de comando
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste( "-f", archivo_kaggle)

  mensaje <- paste0("-m 'envios=", envios,
  "  semilla=", PARAM$semilla_primigenia,
    "'" )

  linea <- paste( comando, competencia, arch, mensaje)

  salida <- system(linea, intern=TRUE) # el submit a Kaggle
  cat(salida, "\n")
}

Successfully submitted to Labol 2025 Virtual analista jr 
Successfully submitted to Labol 2025 Virtual analista jr 
Successfully submitted to Labol 2025 Virtual analista jr 
Successfully submitted to Labol 2025 Virtual analista jr 
Successfully submitted to Labol 2025 Virtual analista jr 
Successfully submitted to Labol 2025 Virtual analista jr 
Successfully submitted to Labol 2025 Virtual analista jr 


In [71]:
# grabo los parametros
if( !require("yaml")) install.packages("yaml")
require("yaml")

write_yaml( PARAM, file="PARAM.yml")

Loading required package: yaml



In [72]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Nov 30 12:57:31 2025"

In [73]:
# ------------------------------------------
#  Ahora también lo guardo como CSV
# ------------------------------------------

# Paso 1: convertir PARAM (que es una lista grande y anidada) en algo "plano"
#         para poder escribir un CSV.

param_flat <- as.data.frame(
  t( unlist(PARAM) ),
  stringsAsFactors = FALSE
)

# Nombre del archivo CSV
file_name_csv <- paste0(
  "PARAM_",
  PARAM$semilla_primigenia,
  PARAM$experimento,
  ".csv"
)

# Paso 2: guardarlo
write.csv(param_flat, file = file_name_csv, row.names = FALSE)